In [1]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score, classification_report, log_loss
import matplotlib.pyplot as plt

data = pd.read_excel('D:\Liberty 2024 Base\Actual_Data_South.xlsx')

<>:9: SyntaxWarning: invalid escape sequence '\L'
<>:9: SyntaxWarning: invalid escape sequence '\L'
C:\Users\Admin\AppData\Local\Temp\ipykernel_15344\2654587516.py:9: SyntaxWarning: invalid escape sequence '\L'
  data = pd.read_excel('D:\Liberty 2024 Base\Actual_Data_South.xlsx')


In [2]:
selected_columns = ['Policy No',  'Renewal Type', 'Product name ', 'Product name  2', 'biztype', 'Policy End Date', 'Policy Start Date',
                    'Reg no ', 'age', 'MANUFACTURER/Make', 'model', 'variant', 'Fuel Type', 'RTO Location ', 'Before GST Add-on GWP',  
                    'Total OD Premium', 'Total TP Premium', 'gst', 'Total Premium Payable ', 'NCB % Previous Year', 'Vehicle Segment', 
                    'Applicable Discount with NCB', 'New Branch Name  2', 'decline', 'Tie Up', 'state2', 'Cleaned_Insured name', 
                    'CustomerID', 'Policy Status', 'Policy Tenure', 'Customer Tenure', 'New Customers', 
                    'Renewal Rate Status', 'Claim in last year', 'Number of Claims']

data = data[selected_columns]

import pandas as pd

# Convert 'Policy End Date' column to datetime format
data['Policy End Date'] = pd.to_datetime(data['Policy End Date'], errors='coerce')

# Step 1: Extract open customers for October 2024
oct_2024_mask = (
    (data['Policy End Date'].dt.year == 2024) & 
    (data['Policy End Date'].dt.month == 10)
)

open_customers = data[oct_2024_mask].copy()

# Step 2: Filter main data for 'Renewed' and 'Not Renewed' statuses up to September 2024
renewed_mask = data[data['Policy Status'].isin(['Renewed', 'Not Renewed'])]
sep_2024_mask = (data['Policy End Date'] <= pd.to_datetime('2024-09-30'))

# Apply the filter and keep only the records with 'Renewed' or 'Not Renewed' statuses up to September 2024
data = data[renewed_mask & sep_2024_mask].copy()

# Convert the 'Churn Label' to binary format (1 for 'Yes', 0 for 'No')
data['Policy Status'] = data['Policy Status'].apply(lambda x: 1 if x == 'Not Renewed' else 0)

for column in data.columns:
    if data[column].dtype == 'object':
        data[column] = data[column].fillna('missing')
    else:
        data[column] = data[column].fillna(0)

date_columns = ['Policy Start Date', 'Policy End Date']

for col in date_columns:
    data[col] = pd.to_datetime(data[col], errors='coerce')  

new_date_cols = {}
for col in date_columns:
    new_date_cols[f'{col}_YEAR'] = data[col].dt.year
    new_date_cols[f'{col}_MONTH'] = data[col].dt.month
    new_date_cols[f'{col}_DAY'] = data[col].dt.day

data = pd.concat([data, pd.DataFrame(new_date_cols)], axis=1)

data = data.drop(columns=date_columns)

# Define features for training
features = [col for col in data.columns if col not in ['Policy Status']]

X = data[features]
y = data['Policy Status']

MemoryError: Unable to allocate 1.51 TiB for an array with shape (465220, 445367) and data type float64

In [ ]:
#label encoding for the actual data
label_encoders = {}
for column in X.columns:
    if X[column].dtype == 'object':
        label_encoder = LabelEncoder()
        X[column] = label_encoder.fit_transform(X[column].astype(str))  
        label_encoders[column] = label_encoder  

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, log_loss, confusion_matrix

import xgboost as xgb
from sklearn.metrics import accuracy_score, log_loss, roc_auc_score, classification_report, confusion_matrix, roc_curve
import matplotlib.pyplot as plt

model = xgb.XGBClassifier(
    max_depth=6,
    learning_rate=0.1,
    n_estimators=100,
    random_state=42
)

#Fit the model 
model.fit(X, y)

for col in date_columns:
    if col in open_customers.columns:
        open_customers[col] = pd.to_datetime(open_customers[col], errors='coerce')
    else:
        print(f"Warning: Column {col} not found in open_customers data")

open_customers_new_date_cols = {}
for col in date_columns:
    if col in open_customers.columns:
        open_customers_new_date_cols[f'{col}_YEAR'] = open_customers[col].dt.year
        open_customers_new_date_cols[f'{col}_MONTH'] = open_customers[col].dt.month
        open_customers_new_date_cols[f'{col}_DAY'] = open_customers[col].dt.day

if open_customers_new_date_cols:
    open_customers = pd.concat([open_customers, pd.DataFrame(open_customers_new_date_cols)], axis=1)

open_customers = open_customers.drop(columns=[col for col in date_columns if col in open_customers.columns])

for column in open_customers.columns:
    if open_customers[column].dtype == 'object':
        open_customers[column] = open_customers[column].fillna('missing')
    else:
        open_customers[column] = open_customers[column].fillna(0)

open_customers_encoded = open_customers.copy()

#Label Encoding for open customers
for column in open_customers_encoded.columns:
    if column in label_encoders:  
        encoder = label_encoders[column]

        mapping_dict = {label: i for i, label in enumerate(encoder.classes_)}
        next_unique_value = [max(mapping_dict.values()) + 1]  

        def encode_test_value(value):
            if value in mapping_dict:
                return mapping_dict[value]
            else:
                mapping_dict[value] = next_unique_value[0]
                next_unique_value[0] += 1
                return mapping_dict[value]
        
        open_customers_encoded[column] = open_customers_encoded[column].apply(encode_test_value)

from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, log_loss, confusion_matrix

# Predict the open customers
X_open_customers = open_customers_encoded[features]
y_open_pred = model.predict(X_open_customers)
y_open_pred_proba = model.predict_proba(X_open_customers)[:, 1]

# Add predictions to the open_customers dataframe
open_customers['Predicted Churn Label'] = ['Not Renewed' if pred == 1 else 'Renewed' for pred in y_open_pred]

# Print prediction counts
print(f"Predicted Renewed: {(y_open_pred == 0).sum()}")
print(f"Predicted Not Renewed: {(y_open_pred == 1).sum()}")

# Evaluate metrics for open customers
if 'True Label' in open_customers.columns:  # Assuming 'True Label' column is available
    y_open_true = open_customers['True Label']
    
    open_accuracy = accuracy_score(y_open_true, y_open_pred)  # Overall accuracy
    open_log_loss = log_loss(y_open_true, y_open_pred_proba)  # Log loss
    open_roc_auc = roc_auc_score(y_open_true, y_open_pred_proba)  # ROC AUC
    open_report = classification_report(y_open_true, y_open_pred)  # Classification report
    
    conf_matrix_open = confusion_matrix(y_open_true, y_open_pred)
    
    # Calculate class-wise accuracy
    class_0_accuracy_open = (
        conf_matrix_open[0, 0] / conf_matrix_open[0].sum()
        if conf_matrix_open[0].sum() > 0 else 0
    )  # Class 0 accuracy
    class_1_accuracy_open = (
        conf_matrix_open[1, 1] / conf_matrix_open[1].sum()
        if conf_matrix_open[1].sum() > 0 else 0
    )  # Class 1 accuracy
    
    # Print open customer metrics
    print(f"Open Customer Overall Accuracy: {open_accuracy}")
    print(f"Open Customer Log Loss: {open_log_loss}")
    print(f"Open Customer ROC AUC: {open_roc_auc}")
    print(f"Open Customer Classification Report:\n{open_report}")
    print(f"Class 0 Open Accuracy: {class_0_accuracy_open}")
    print(f"Class 1 Open Accuracy: {class_1_accuracy_open}")

# Evaluate the model on training data
y_pred = model.predict(X)
y_pred_proba = model.predict_proba(X)[:, 1]

train_accuracy = accuracy_score(y, y_pred)
train_log_loss = log_loss(y, y_pred_proba)
train_roc_auc = roc_auc_score(y, y_pred_proba)
train_report = classification_report(y, y_pred)

conf_matrix_train = confusion_matrix(y, y_pred)
class_0_accuracy_train = conf_matrix_train[0, 0] / conf_matrix_train[0].sum()
class_1_accuracy_train = conf_matrix_train[1, 1] / conf_matrix_train[1].sum()

# Print training metrics
print(f"Train Accuracy: {train_accuracy}")
print(f"Train Log Loss: {train_log_loss}")
print(f"Train ROC AUC: {train_roc_auc}")
print(f"Train Classification Report:\n{train_report}")
print(f"Class 0 Train Accuracy: {class_0_accuracy_train}")
print(f"Class 1 Train Accuracy: {class_1_accuracy_train}")

# Save open customer predictions to CSV
open_customers.to_csv('customers_predictions (South) oct.csv', index=False)

In [ ]:
X_open_customers.columns.values

In [6]:
X.to_csv('X test data.csv', index=False)